library imports

In [4]:
import tensorflow as tf
import numpy as np
import os
import time

produce text

In [ ]:
from scraper import S
s = S()
s.make_query()

explore text

In [1]:
# Read, then decode for py2 compat.
text = open('raw.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 95812 characters


first characters

In [3]:
print(text[:250])

nd outdoor, tease, indulge, and play harmless pranks on each other. The acquisition of love and attention itself may be part of the game.[6]
Ludic lovers want to have as much fun as possible. When they are not seeking a stable relationship, they rare


get a list of unique characters

In [ ]:
# The unique characters in the file
unique = sorted(set(text))
print(f'{len(unique)} unique characters')

99 unique characters


vectorize text (convert strings to a numerical representation)

In [ ]:
# split text into tokens
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

add stringlookup layer

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(unique), mask_token=None)
    
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[58, 59, 60, 61, 62, 63, 64], [81, 82, 83]]>

initialize layer for reverse operation (vector to text)

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
    
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

join bit chars into strings with the following function:

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

define the inverse operation:

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

vectorize all text

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1171272,), dtype=int64, numpy=array([1, 2, 1, ..., 1, 2, 1])>

dump text into a dataset

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))









C
R
I
M
E


convert individual characters to sequences of size 100

In [ ]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b'\r' b'\n' b'\r' b'\n' b'C' b'R' b'I' b'M' b'E' b' ' b'A' b'N'
 b'D' b' ' b'P' b'U' b'N' b'I' b'S' b'H' b'M' b'E' b'N' b'T' b'\r' b'\n'
 b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'P' b'A' b'R' b'T' b' '
 b'I' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'C' b'H' b'A' b'P'
 b'T' b'E' b'R' b' ' b'I' b'\r' b'\n' b'\r' b'\n' b'O' b'n' b' ' b'a' b'n'
 b' ' b'e' b'x' b'c' b'e' b'p' b't' b'i' b'o' b'n' b'a' b'l' b'l' b'y'
 b' ' b'h' b'o' b't' b' ' b'e' b'v' b'e' b'n' b'i' b'n' b'g' b' ' b'e'
 b'a' b'r' b'l' b'y' b' ' b'i'], shape=(101,), dtype=string)


join sequences back into strings

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\n\r\n\r\nCRIME AND PUNISHMENT\r\n\r\n\r\n\r\n\r\nPART I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\nOn an exceptionally hot evening early i'
b'n July a young man came out of\r\nthe garret in which he lodged in S. Place and walked slowly, as thoug'
b'h\r\nin hesitation, towards K. bridge.\r\n\r\nHe had successfully avoided meeting his landlady on the stair'
b'case. His\r\ngarret was under the roof of a high, five-storied house and was more\r\nlike a cupboard than'
b' a room. The landlady who provided him with garret,\r\ndinners, and attendance, lived on the floor belo'


define 'split_input_target', a function which, at each time step, takes input as the current character and label as the next character

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

map function across our sequences, save to dataset

In [ ]:
dataset = sequences.map(split_input_target)
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\n\r\n\r\nCRIME AND PUNISHMENT\r\n\r\n\r\n\r\n\r\nPART I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\nOn an exceptionally hot evening early '
Target: b'\r\n\r\nCRIME AND PUNISHMENT\r\n\r\n\r\n\r\n\r\nPART I\r\n\r\n\r\n\r\nCHAPTER I\r\n\r\nOn an exceptionally hot evening early i'


shuffle and pack data into batches for training

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

build 3-layer model

In [6]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Batch size
batch_size = 64

NameError: name 'ids_from_chars' is not defined

In [ ]:
from tensorflow.keras import layers

class Model(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units, batch_size):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.rnn_units = rnn_units
        self.embedding = layers.Embedding(vocab_size, embedding_dim)
        self.lstm = layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')
        self.dense = layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
            x, states = self.gru(x, initial_state=states, training=training)
            x = self.dense(x, training=training)

            if return_state:
                return x, states
            else:
                return x

model = Model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [ ]:
# class MyModel(tf.keras.Model):
#   def __init__(self, vocab_size, embedding_dim, rnn_units):
#     super().__init__(self)
#     self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
#     self.gru = tf.keras.layers.GRU(rnn_units,
#                                    return_sequences=True,
#                                    return_state=True)
#     self.dense = tf.keras.layers.Dense(vocab_size)

#   def call(self, inputs, states=None, return_state=False, training=False):
#     x = inputs
#     x = self.embedding(x, training=training)
#     if states is None:
#       states = self.gru.get_initial_state(x)
#     x, states = self.gru(x, initial_state=states, training=training)
#     x = self.dense(x, training=training)

#     if return_state:
#       return x, states
#     else:
#       return x

In [ ]:
# model = MyModel(
#     vocab_size=vocab_size,
#     embedding_dim=embedding_dim,
#     rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 100) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  25600     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  102500    
                                                                 
Total params: 4,066,404
Trainable params: 4,066,404
Non-trainable params: 0
_________________________________________________________________


make predictions by sampling from output distribution

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

sampled_indices

array([ 6, 18, 84, 78, 95, 23, 76, 48,  9,  9, 10, 98, 85, 40, 91, 72, 36,
        2, 29, 49, 99, 91, 49, 96, 45, 29, 77, 60, 92, 80, 43,  0, 17, 22,
       62, 31,  3, 61, 21, 44, 42, 96,  4,  1, 20, 13, 48, 98, 12, 85, 75,
       25, 69, 90, 82, 87, 16, 47,  1, 31,  3,  2, 21, 31, 44, 98, 88, 73,
        4, 40, 44, 99, 94, 57, 23,  1, 97, 75, 59, 96, 70, 81, 94, 54, 21,
       26, 61, 89, 20, 61, 61,  6, 94, 36, 35, 32, 26, 86, 52, 15])

decode

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'\r\nanything more.\xe2\x80\x99 I am telling you Dushkin\xe2\x80\x99s story. \xe2\x80\x98I gave him a note\xe2\x80\x99--a\r\nrouble that is--\xe2\x80\x98for I t'

Next Char Predictions:
 b'$2\xc3\xa0u\xc3\xbc7sT(()\xe2\x80\x9c\xc3\xa4L\xc3\xaeoH\rAU\xe2\x80\x9d\xc3\xaeU\xe2\x80\x98QAtc\xc3\xafwO[UNK]16eC d5PN\xe2\x80\x98!\n4-T\xe2\x80\x9c,\xc3\xa4r9l\xc3\xaay\xc3\xa70S\nC \r5CP\xe2\x80\x9c\xc3\xa8p!LP\xe2\x80\x9d\xc3\xb6_7\n\xe2\x80\x99rb\xe2\x80\x98mx\xc3\xb6Z5:d\xc3\xa94dd$\xc3\xb6HGD:\xc3\xa6X/'


add optimizer, loss function

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 100)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.6051164, shape=(), dtype=float32)


In [ ]:
#should be roughly equal to vocab size, 99
tf.exp(example_batch_mean_loss).numpy()

99.99461

compile model

In [ ]:
model.compile(optimizer='adam', loss=loss)

add checkpoints

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
181/181 [==============================] - 12s 53ms/step - loss: 2.6634
Epoch 2/100
181/181 [==============================] - 11s 54ms/step - loss: 1.9266
Epoch 3/100
181/181 [==============================] - 11s 54ms/step - loss: 1.5932
Epoch 4/100
181/181 [==============================] - 11s 55ms/step - loss: 1.4071
Epoch 5/100
181/181 [==============================] - 12s 55ms/step - loss: 1.3008
Epoch 6/100
181/181 [==============================] - 11s 56ms/step - loss: 1.2297
Epoch 7/100
181/181 [==============================] - 11s 57ms/step - loss: 1.1762
Epoch 8/100
181/181 [==============================] - 11s 57ms/step - loss: 1.1304
Epoch 9/100
181/181 [==============================] - 11s 57ms/step - loss: 1.0884
Epoch 10/100
181/181 [==============================] - 11s 57ms/step - loss: 1.0489
Epoch 11/100
181/181 [==============================] - 11s 57ms/step - loss: 1.0096
Epoch 12/100
181/181 [==============================] - 12s 58ms/step - lo

generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant([''])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

The world seemed like such a peaceful place until the magic tree was discovered in London.

He had not chasted in Loze him extreme coldirm the collection of Polenka and proved it against his
face. He heard their cases to the Project

 God granted, I... and as for your plare you will help out to uttered both hands in
every weeping.). It does it matter of compliance with my sons!”

“Yes, you are sitting on her slip. This murder,
she cried irritating. “It was. I’ve come to be different and refuence,
imaginary men money with me or indinged), but that grewoment, who had
to defe it in your position,” he observed gloomy roubles pension, as though not
understood genially. “I have bound him,
feeling very sensibly, to very requirent facts, the same fuen into a vigorous tymandar; there were
so unexpectedly, terrible, if you expect upon your sense if you are minute, you
were twise...”



CHAPTER IV

Sonia stopped on her derangemptailiand was standing in despair.

“Never,” on him, the daughters of 